In [18]:
import os
import shutil
import re
import pandas as pd

In [19]:
# Define source and destination directories
src_dir = "./Data/Preprocessed"
dst_dir = "./Data/Processed"

# Ensure the destination directory exists
os.makedirs(dst_dir, exist_ok=True)

# Loop through all files in the source directory

for folder in os.listdir(src_dir):
    folder_path = os.path.join(src_dir, folder)
    folder_dst_dir = os.path.join(dst_dir, folder)

    os.makedirs(folder_dst_dir, exist_ok=True)

    if not os.path.isdir(folder_path):
        continue

    for filename in os.listdir(folder_path):
        if "filtered" in filename:
            continue

        if 'Ai213' in filename and ('.CSV' in filename or '.csv' in filename):

            start_index = filename.find('Ai213')
            # Remove all characters before 'Ai213' and change .CSV to .csv
            new_filename = filename[start_index:].replace('.CSV', '.csv')

            # Extract the grouping pattern Ai213_x=x_#x using regex
            match = re.search(r'Ai213_\d+-\d+_#\d+', new_filename)
            if match:
                if "Side_viewDLC_Resnet50" in filename:
                    new_filename = match.group(0) + "_Pose_Data.csv"
                
                # Create a subdirectory based on the matched pattern
                subfolder_name = match.group(0)
                subfolder_path = os.path.join(folder_dst_dir, subfolder_name)
                os.makedirs(subfolder_path, exist_ok=True)

                # Construct full file paths
                old_filepath = os.path.join(folder_path, filename)
                new_filepath = os.path.join(subfolder_path, new_filename)

                # Move and rename the file
                shutil.copy(old_filepath, new_filepath)

    print("Files have been renamed, grouped into folders, and moved to the new directory.")


Files have been renamed, grouped into folders, and moved to the new directory.
Files have been renamed, grouped into folders, and moved to the new directory.
Files have been renamed, grouped into folders, and moved to the new directory.
Files have been renamed, grouped into folders, and moved to the new directory.


In [20]:
def reduce_df(df_path):
    df = pd.read_csv(df_path, index_col=0)
    likelihood_cols = [col for col in df.columns if 'likelihood' in col]
    df['total_likelihood'] = df[likelihood_cols].sum(axis=1)

    # reduced_df = df.groupby(df.index // 8).first().reset_index(drop=True)
    # Group rows into groups of 8 and select the one with the highest total likelihood in each group
    grouped = df.groupby(df.index // 8)

    reduced_df = grouped.apply(lambda group: group.loc[group['total_likelihood'].idxmax()])

    bool_columns = ['Is_Voiding', 'Shock_Start', 'Shock_End', 'Tone_Start', 'Tone_End']
    for col in bool_columns:
        reduced_df[col] = grouped[col].any()

    # Drop the 'total_likelihood' column used for selection
    reduced_df = reduced_df.drop(columns=['total_likelihood'])
    reduced_df = reduced_df.reset_index(drop=True)
    reduced_df.to_csv(df_path)

In [21]:
def fix_void_timing(df_path):
    df = pd.read_csv(df_path)
    df['seconds'] = df['Var4'].str.extract(r'(\d+\.\d+)')
    df = df[['seconds']]
    df.to_csv(df_path)

In [22]:
def fix_pose_data(df_path):
    df = pd.read_csv(df_path)

    body_parts = df.iloc[0, 1:] 
    coords = df.iloc[1, 1:] 

    # Initialize an empty list to hold the new column names
    new_columns = []
    new_columns.append(f'Image')
    # Create new column names based on body parts and coordinates
    for part, coord in zip(body_parts, coords):
        new_columns.append(f'{part}_{coord}')
    df.columns = new_columns
    df = df[2:]
    df = df.reset_index(drop=True)
    df = df.drop(['Image'], axis=1)

    df.to_csv(df_path)
    # df_drop = df[[col for col in df.columns if col.endswith('_x') or col.endswith('_y')]]

In [23]:
def fix_time_df(df_path):
    df = pd.read_csv(df_path, header=None, names=['DateTime', 'Seconds'])
    df = df.replace(r'[()]', '', regex=True)
    df.to_csv(df_path)

In [24]:
def combine_dfs(pose_path, side_path, void_path, shock_on_path, shock_off_path, tone_on_path, tone_off_path, new_path):
    pose_data_df = pd.read_csv(pose_path, index_col=0)
    side_view_df = pd.read_csv(side_path , index_col=0)
    void_data_df = pd.read_csv(void_path , index_col=0)
    shock_on_df = pd.read_csv(shock_on_path, index_col=0)
    shock_off_df = pd.read_csv(shock_off_path, index_col=0)
    tone_on_df = pd.read_csv(tone_on_path, index_col=0)
    tone_off_df = pd.read_csv(tone_off_path, index_col=0)

    pose_time_df = pd.merge(pose_data_df, side_view_df, left_index=True, right_index=True)
    pose_time_df['Is_Voiding'] = False 

    for voidtime in void_data_df["seconds"]:
        pose_time_df['difference'] = (pose_time_df['Seconds'] - voidtime).abs()

        closest_index = pose_time_df['difference'].idxmin()

        pose_time_df.loc[closest_index, 'Is_Voiding'] = True
        pose_time_df = pose_time_df.drop(columns=["difference"])

    pose_time_df['Shock_Start'] = False 
    for shock_on in shock_on_df["side_Shock_frame"]:

        pose_time_df.loc[shock_on, 'Shock_Start'] = True
        
    pose_time_df['Shock_End'] = False 
    for shock_off in shock_off_df["side_Shock_frame"]:

        pose_time_df.loc[shock_off, 'Shock_End'] = True

    pose_time_df['Tone_Start'] = False 
    for tone_on in tone_on_df["side_Tone_frame"]:

        pose_time_df.loc[tone_on, 'Tone_Start'] = True

    pose_time_df['Tone_End'] = False 
    for tone_off in tone_off_df["side_Tone_frame"]:

        pose_time_df.loc[tone_off, 'Tone_End'] = True

    pose_time_df.to_csv(new_path)

In [ ]:
for trial in os.listdir(dst_dir):
    trail_path = os.path.join(dst_dir, trial)
    
    folders = [os.path.join(trail_path, d) for d in os.listdir(trail_path) if os.path.isdir(os.path.join(trail_path, d))]
    for folder in folders:
        
        for filename in os.listdir(folder):
            if "Bottom_camera" in filename:
                bottom_path = os.path.join(folder,filename)

            if "Pose_Data" in filename:
                pose_path = os.path.join(folder,filename)

            if "ShockOffset" in filename:
                shock_off_path = os.path.join(folder,filename)

            if "ShockONset" in filename:
                shock_on_path = os.path.join(folder,filename)

            if "Side_view" in filename:
                side_path = os.path.join(folder,filename)

            if "ToneOffset" in filename:
                tone_off_path = os.path.join(folder,filename)

            if "ToneONset" in filename:
                tone_on_path = os.path.join(folder,filename)

            if "VoidTiming" in filename:
                void_path = os.path.join(folder,filename)



        fix_pose_data(pose_path)
        fix_void_timing(void_path)
        fix_time_df(bottom_path)
        fix_time_df(side_path)

        new_path = os.path.join(folder, "pose_void_tone_shock_combined.csv")
        combine_dfs(pose_path, side_path, void_path, shock_on_path, shock_off_path, tone_on_path, tone_off_path, new_path)
        reduce_df(new_path)


/tmp/ipykernel_18626/2879870463.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)
/tmp/ipykernel_18626/2879870463.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)
/tmp/ipykernel_18626/2879870463.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_cs